# 运行时错误

In [1]:
import tvm
from tvm import te
import tvm.testing

## 测试 op 翻译

In [2]:
ferror = tvm.testing.test_raise_error_callback("OpNotImplemented: myop")

In [4]:
try:
    ferror()
    assert False
except tvm.error.OpNotImplemented as e:
    msg = str(e)
    assert isinstance(e, NotImplementedError)
    assert msg.find("ffi_testing.cc") != -1

In [7]:
fchk_eq = tvm.testing.test_check_eq_callback("InternalError: myop")
try:
    fchk_eq(0, 1)
    assert False
except tvm.error.InternalError as e:
    msg = str(e)
    assert msg.find("ffi_testing.cc") != -1

In [8]:
try:
    tvm.testing.ErrorTest(0, 1)
    assert False
except ValueError as e:
    msg = str(e)
    assert msg.find("ffi_testing.cc") != -1

# 测试深度回调

In [9]:
def test_deep_callback():
    def error_callback():
        raise ValueError("callback error")

    wrap1 = tvm.testing.test_wrap_callback(error_callback)

    def flevel2():
        wrap1()

    wrap2 = tvm.testing.test_wrap_callback(flevel2)

    def flevel3():
        wrap2()

    wrap3 = tvm.testing.test_wrap_callback(flevel3)

    try:
        wrap3()
        assert False
    except ValueError as e:
        msg = str(e)
        idx2 = msg.find("in flevel2")
        idx3 = msg.find("in flevel3")
        assert idx2 != -1 and idx3 != -1
        assert idx2 > idx3

In [10]:
test_deep_callback()

In [13]:
x = tvm.ir.make_node("IntImm", dtype="int32", value=10)
assert isinstance(x, tvm.tir.IntImm)
assert x.value == 10

TVMError: Traceback (most recent call last):
  6: TVMFuncCall
  5: _ZN3tvm7runtime13PackedFu
  4: tvm::MakeNode(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)
  3: tvm::ReflectionVTable::CreateObject(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::TVMArgs const&)
  2: tvm::InitNodeByPackedArgs(tvm::ReflectionVTable*, tvm::runtime::Object*, tvm::runtime::TVMArgs const&)
  1: tvm::NodeAttrSetter::Visit(char const*, tvm::runtime::ObjectRef*)
  0: tvm::NodeAttrSetter::GetAttr(char const*) [clone .part.0]
  File "/media/pc/data/lxw/ai/tvm/src/node/reflection.cc", line 189
IntImm: require field span

In [14]:
x = tvm.ir.make_node?

Signature: tvm.ir.make_node(type_key, **kwargs)
Docstring:
Make a new IR node by its type key and fields

Parameters
----------
type_key : str
    The type key of the node.

**kwargs : dict
    The fields of the node.

Returns
-------
node : Node
    The corresponding IR Node

Note
----
If the created node is instance of AttrsNode, then
the creator function will also run bound checks and
default value setup as supported by Attrs.

Example
-------
The following code constructs a IntImm object

.. code-block:: python

   x = tvm.ir.make_node("IntImm", dtype="int32", value=10)
   assert isinstance(x, tvm.tir.IntImm)
   assert x.value == 10
File:      /media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/ir/attrs.py
Type:      function